# OpenStreetMap Data Case Study

### Map Area

San Jose, CA, United States

- https://mapzen.com/data/metro-extracts/ (Metro Extract has been discontinued as of Feb 1st, 2018)
- https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md (Sample SQL project)
- https://docs.python.org/3/library/sqlite3.html (Python documentation)

## Problems Encountered in the Map

In [1]:
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re

I first run the code to find out how many different types of tags are there and the count of each tag.

It helps to provide an overview of the amount of data in the file.

In [2]:
'''
The code below is to find out how many types of tags are there and the number of each tag.
'''

def count_tags(filename):
    tags = {}
    for event, element in ET.iterparse(filename):
        if element.tag not in tags.keys():
            tags[element.tag] = 1
        else:
            tags[element.tag] += 1
    return tags

def test():

    tags = count_tags('san-jose_california.osm')
    pprint.pprint(tags)

if __name__ == "__main__":
    test()

{'bounds': 1,
 'member': 22273,
 'nd': 2217224,
 'node': 1898718,
 'osm': 1,
 'relation': 2642,
 'tag': 780009,
 'way': 250204}


Then, I check for any errors in the tags by classifying them into tags with:

- only lowercase
- lowercase and colon
- problematic characters
- others

In [3]:
'''
The code below allows you to check the k value for each tag.
By classifying the tagss into few categories:
1. "lower": valid tags containing only lowercase letters
2. "lower_colon": valid tags with a colon in the names
3. "problemchars": tags with problematic characters
4. "other": other tags that don't fall into the 3 categories above
'''
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if re.search(lower,k):
            keys["lower"] += 1
        elif re.search(lower_colon,k):
            keys["lower_colon"] += 1
        elif re.search(problemchars,k):
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

def test():
    keys = process_map('san-jose_california.osm')
    pprint.pprint(keys)

if __name__ == "__main__":
    test()

{'lower': 522227, 'lower_colon': 235222, 'other': 22560, 'problemchars': 0}


### Auditng Street Names

I construct of a list of common expected street type such as "Street", "Avenue" etc.

Then, I list out all the street type not in the expected street type

In [4]:
'''
The code below lists out all the street types not in the expected list.
'''
OSMFILE = "san-jose_california.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "Terrace", "Way"]


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

st_types = audit(OSMFILE)

def test():    
    pprint.pprint(dict(st_types))

if __name__ == '__main__':
    test()

{'0.1': set(['Ala 680 PM 0.1']),
 '1': set(['Prospect Rd #1', 'Stewart Drive Suite #1']),
 '114': set(['West Evelyn Avenue Suite #114']),
 '201': set(['Great America Pkwy Ste 201']),
 '4A': set(['Saratoga Avenue Bldg 4A']),
 '6': set(['Martin Avenue #6', 'Pruneridge Ave #6']),
 '7.1': set(['Hwy 17 PM 7.1']),
 '81': set(['Concourse Dr #81']),
 'Alameda': set(['The Alameda']),
 'Alley': set(['Fountain Alley']),
 'Ave': set(['1425 E Dunne Ave',
             'Blake Ave',
             'Cabrillo Ave',
             'Cherry Ave',
             'E Duane Ave',
             'Foxworthy Ave',
             'Greenbriar Ave',
             'Hillsdale Ave',
             'Hollenbeck Ave',
             'Meridian Ave',
             'N Blaney Ave',
             'Saratoga Ave',
             'Seaboard Ave',
             'The Alameda Ave',
             'W Washington Ave',
             'Walsh Ave',
             'Westfield Ave']),
 'Barcelona': set(['Calle de Barcelona']),
 'Bascom': set(['S. Bascom']),
 'Bellomy

It can be seen from the output above that there are a lot of inconsistencies and error in street type.

Few examples are:
- "St" (overabbreviated)
- "4A" (referring to unit number)
- "Alameda" (building name)
- "CA" (state name)

While it will be difficult to clean most inconsistencies in street type, the overabbreviated ones are easier to identify and correct. A mapping list is written by pairing the overabbreviated name to the full street type name.

In [5]:
'''
The code below updates the unexpected street types listed in the mapping list
while keeping others unchanged.
'''
mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Blvd": "Boulevard",
            "Dr": "Drive",
            "Rd": "Road"
            }

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m.group() not in expected:
        if m.group() in mapping.keys():
            name = re.sub(m.group(), mapping[m.group()], name)
    return name

def test():
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name

if __name__ == '__main__':
    test()

Martin Avenue #6 => Martin Avenue #6
Pruneridge Ave #6 => Pruneridge Ave #6
Concourse Dr #81 => Concourse Dr #81
Winchester => Winchester
Gaundabert Ln => Gaundabert Ln
Barber Ln => Barber Ln
Branham Ln => Branham Ln
Park Circle West => Park Circle West
Vanderbilt Court West => Vanderbilt Court West
Wolfe Rd => Wolfe Road
Mt Hamilton Rd => Mt Hamilton Road
Berryessa Rd => Berryessa Road
Saratoga Los Gatos Rd => Saratoga Los Gatos Road
Quimby Rd => Quimby Road
San Antonio Valley Rd => San Antonio Valley Road
Homestead Rd => Homestead Road
Mt. Hamilton Rd => Mt. Hamilton Road
Silver Creek Valley Rd => Silver Creek Valley Road
West Evelyn Avenue Suite #114 => West Evelyn Avenue Suite #114
Hwy 17 PM 7.1 => Hwy 17 PM 7.1
Blossom Hill => Blossom Hill
wilcox ave => wilcox ave
North Fair Oaks => North Fair Oaks
Vanderbilt Court East => Vanderbilt Court East
Park Circle East => Park Circle East
Rio Robles East => Rio Robles East
The Alameda => The Alameda
yes => yes
Monterey Highway => Monterey

### Auditng Zip Code

In [6]:
'''
This code checks for zipcode whether they begin with '94' or '95' or something else
'''
OSMFILE = "san-jose_california.osm"
zip_type_re = re.compile(r'\d{5}$')

def audit_ziptype(zip_types, zipcode):
    if zipcode[0:2] != 95:
        zip_types[zipcode[0:2]].add(zipcode)
    elif zipcode[0:2] != 94:
        zip_types[zipcode[0:2]].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    zip_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_ziptype(zip_types,tag.attrib['v'])
    osm_file.close()
    return zip_types

zip_print = audit_zip(OSMFILE)

def test():    
    pprint.pprint(dict(zip_print))

if __name__ == '__main__':
    test()

Some results have been abbreviated
{'94': set(['94024',
            '94084',
            '94085',
            '94085-3010',
            '94086',
            '94086-6406',
            '94087',
            u'94087\u200e',
            '94088-3453',
            '94088-3707',
            '94089',
            '94089-2701',
            '94538',
            '94807']),
 '95': set(['95002',
            '95008',
            '95013',
            '95014',
            '95014-0200',
            '95014-0202',
            '95014-0236',
            '95134',
            '95134-1358',
            '95135',
            '95136',
            '95148',
            '95191',
            '95914']),
 'CA': set(['CA 94035',
            'CA 94085',
            'CA 94086',
            'CA 95054',
            'CA 95110',
            'CA 95116']),
 'CU': set(['CUPERTINO'])}


update the zip code

In [7]:
'''
This code will update non 5-digit zipcode.
If it is 8/9-digit, only the first 5 digits are kept.
If it has the state name in front, only the 5 digits are kept.
If it is something else, will not change anything as it might result in error when validating the csv file.
'''
def update_zipcode(zipcode):
    """Clean postcode to a uniform format of 5 digit; Return updated postcode"""
    if re.findall(r'^\d{5}$', zipcode): # 5 digits 02118
        valid_zipcode = zipcode
        return valid_zipcode
    elif re.findall(r'(^\d{5})-\d{3}$', zipcode): # 8 digits 02118-029
        valid_zipcode = re.findall(r'(^\d{5})-\d{3}$', zipcode)[0]
        return valid_zipcode
    elif re.findall(r'(^\d{5})-\d{4}$', zipcode): # 9 digits 02118-0239
        valid_zipcode = re.findall(r'(^\d{5})-\d{4}$', zipcode)[0]
        return valid_zipcode
    elif re.findall(r'CA\s*\d{5}', zipcode): # with state code CA 02118
        valid_zipcode =re.findall(r'\d{5}', zipcode)[0]  
        return valid_zipcode  
    else: #return default zipcode to avoid overwriting
        return zipcode
    
def test_zip():
    for zips, ways in zip_print.iteritems():
        for name in ways:
            better_name = update_zipcode(name)
            print name, "=>", better_name

if __name__ == '__main__':
    test_zip()

Some results have been abbreviated
CA 95054 => 95054
CA 94035 => 94035
CA 95110 => 95110
CA 95116 => 95116
CA 94085 => 94085
CA 94086 => 94086
95014-1899 => 95014
95014-5398 => 95014
95014-3456 => 95014
95032-2502 => 95032
94087 => 94087
94024 => 94024
94087‎ => 94087‎
94089-2701 => 94089
94538 => 94538
94085-3010 => 94085
94086-6406 => 94086
94088-3707 => 94088
CUPERTINO => CUPERTINO


## Convert Database into SQL

5 cvs files will be created using the schema below which will be then converted into an SQL databases.

In [8]:
import csv
import codecs
import cerberus
import schema

In [9]:
# %load schema.py
'''
The schema below for the 5 csv files which will be used to construct a SQL databases
'''

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


5 cvs files are created with the scheme as above.

In [10]:
'''
The code below is mostly derived from Udacity Lession 13: Case study: OpenStreetMap Data [SQL]
https://classroom.udacity.com/nanodegrees/nd002/parts/860b269a-d0b0-4f0c-8f3d-ab08865d43bf/modules/316820862075461/lessons/5436095827/concepts/54908788190923
'''
OSM_PATH = "san-jose_california.osm"
OSMFILE = "san-jose_california.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "Terrace", "Way"]

mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Blvd": "Boulevard",
            "Dr": "Drive",
            "Rd": "Road"
            }

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']
    
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    p=0
    
    if element.tag == 'node':
        for i in NODE_FIELDS:
            node_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):
            node_tags_attribs = {}
            temp = LOWER_COLON.search(tag.attrib['k'])
            is_p = PROBLEMCHARS.search(tag.attrib['k'])
            if is_p:
                continue
            elif temp:
                split_char = temp.group(1)
                split_index = tag.attrib['k'].index(split_char)
                type1 = temp.group(1)
                node_tags_attribs['id'] = element.attrib['id']
                node_tags_attribs['key'] = tag.attrib['k'][split_index+2:]
                node_tags_attribs['value'] = tag.attrib['v']
                node_tags_attribs['type'] = tag.attrib['k'][:split_index+1]
                if node_tags_attribs['type'] == "addr" and node_tags_attribs['key'] == "street":
                    # update street name
                    node_tags_attribs['value'] = update_name(tag.attrib['v'], mapping) 
                #elif node_tags_attribs['type'] == "addr" and node_tags_attribs['key'] == "postcode":
                #    # update post code
                #    node_tags_attribs['value'] = update_zipcode(tag.attrib['v']) 
            else:
                node_tags_attribs['id'] = element.attrib['id']
                node_tags_attribs['key'] = tag.attrib['k']
                node_tags_attribs['value'] = tag.attrib['v']
                node_tags_attribs['type'] = 'regular'
                if node_tags_attribs['type'] == "addr" and node_tags_attribs['key'] == "street":
                    # update street name
                    node_tags_attribs['value'] = update_name(tag.attrib['v'], mapping) 
                #elif node_tags_attribs['type'] == "addr" and node_tags_attribs['key'] == "postcode":
                #    # update post code
                #    node_tags_attribs['value'] = update_zipcode(tag.attrib['v']) 
            tags.append(node_tags_attribs)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        id = element.attrib['id']
        for i in WAY_FIELDS:
            way_attribs[i] = element.attrib[i]
        for i in element.iter('nd'):
            d = {}
            d['id'] = id
            d['node_id'] = i.attrib['ref']
            d['position'] = p
            p+=1
            way_nodes.append(d)
        for c in element.iter('tag'):
            temp = LOWER_COLON.search(c.attrib['k'])
            is_p = PROBLEMCHARS.search(c.attrib['k'])
            e = {}
            if is_p:
                continue
            elif temp:
                split_char = temp.group(1)
                split_index = c.attrib['k'].index(split_char)
                e['id'] = id
                e['key'] = c.attrib['k'][split_index+2:]
                e['type'] = c.attrib['k'][:split_index+1]
                e['value'] = c.attrib['v']
                if e['type'] == "addr" and e['key'] == "street":
                    e['value'] = update_name(c.attrib['v'], mapping) 
                #elif e['type'] == "addr" and e['key'] == "postcode":
                #    e['value'] = update_zipcode(c.attrib['v'])
            else:
                e['id'] = id
                e['key'] = c.attrib['k']
                e['type'] = 'regular'
                e['value'] =  c.attrib['v']
                if e['type'] == "addr" and e['key'] == "street":
                    e['value'] = update_name(c.attrib['v'], mapping) 
                #elif e['type'] == "addr" and e['key'] == "postcode":
                #    e['value'] = update_zipcode(c.attrib['v'])
            tags.append(e)
        
    return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
        
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""
    with codecs.open(NODES_PATH, 'wb') as nodes_file, \
        codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, \
        codecs.open(WAYS_PATH, 'wb') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, \
        codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

The 5 csv files are then converted into a .db database using the code below.

In [11]:
# Creating the database and tables
import sqlite3
conn = sqlite3.connect('data_wrangling.sqlite')

In [12]:
conn.text_factory = str
cur = conn.cursor()

#Make some fresh tables using executescript()
cur.execute('''DROP TABLE IF EXISTS nodes''')
cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
cur.execute('''DROP TABLE IF EXISTS ways''')
cur.execute('''DROP TABLE IF EXISTS ways_tags''')
cur.execute('''DROP TABLE IF EXISTS ways_nodes''')


cur.execute('''CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT)
''')

with open('nodes.csv','r') as nodes_table: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(nodes_table) # comma is default delimiter
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO nodes VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)


cur.execute('''CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id))
''')

with open('nodes_tags.csv','r') as nodes_tags_table: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(nodes_tags_table) # comma is default delimiter
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags VALUES (?, ?, ?, ?);", to_db)

cur.execute('''CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT)
''')

with open('ways.csv','r') as ways_table: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(ways_table) # comma is default delimiter
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways VALUES (?, ?, ?, ?, ?, ?);", to_db)

cur.execute('''CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id))
''')

with open('ways_tags.csv','r') as ways_tags_table: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(ways_tags_table) # comma is default delimiter
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags VALUES (?, ?, ?, ?);", to_db)

cur.execute('''CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id))
''')

with open('ways_nodes.csv','r') as ways_nodes_table: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(ways_nodes_table) # comma is default delimiter
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes VALUES (?, ?, ?);", to_db)

#Save changes
conn.commit()

## Data Overviews

The queries below provides an overview of the San Jose, CA OpenStreetMap dataset.

### Files size

In [13]:
import os

print ('The san-jose_california.osm file is {} MB'.format(os.path.getsize('san-jose_california.osm')/1.0e6))
print ('The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print ('The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes_tags.csv')/1.0e6))
print ('The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/1.0e6))
print ('The ways_nodes.csv file is {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6))
print ('The ways_tags.csv file is {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6))
print ('The data_wrangling.sqlite file is {} MB'.format(os.path.getsize('data_wrangling.sqlite')/1.0e6))

The san-jose_california.osm file is 412.959202 MB
The nodes.csv file is 160.198493 MB
The nodes_tags.csv file is 3.220635 MB
The ways.csv file is 15.040047 MB
The ways_nodes.csv file is 52.909335 MB
The ways_tags.csv file is 23.625255 MB
The data_wrangling.sqlite file is 226.950144 MB


### Number of nodes

In [14]:
nodes_count = conn.cursor()
nodes_count.execute("SELECT COUNT(*) FROM nodes")

nodes_count = nodes_count.fetchall()
 
for row in nodes_count:
    print(row)

(1898718,)


### Number of ways

In [15]:
ways_count = conn.cursor()
ways_count.execute("SELECT COUNT(*) FROM ways")

ways_count = ways_count.fetchall()
 
for row in ways_count:
    print(row)

(250204,)


### Number of unique users

In [16]:
unique_user = conn.cursor()
unique_user.execute("SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e")

unique_user = unique_user.fetchall()
 
for row in unique_user:
    print(row)

(1480,)


### Top 5 user

In [17]:
top_user = conn.cursor()
top_user.execute("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC LIMIT 5")

top_user = top_user.fetchall()
 
for row in top_user:
    print(row)

('andygol', 295700)
('nmixter', 281943)
('mk408', 134593)
('Bike Mapper', 94851)
('samely', 80727)


## Data Exploration

### Top 10 Amenities

In [18]:
top_amenities = conn.cursor()
t = ('amenity',)
top_amenities.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC LIMIT 10", t)

top_amenities = top_amenities.fetchall()
 
for row in top_amenities:
    print(row)

('restaurant', 883)
('fast_food', 429)
('bench', 320)
('cafe', 262)
('bicycle_parking', 210)
('place_of_worship', 170)
('toilets', 163)
('school', 137)
('fuel', 130)
('bank', 129)


### Most Popular Sports

In [19]:
top_sports = conn.cursor()
t = ('sport',)
top_sports.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC LIMIT 10", t)

top_sports = top_sports.fetchall()
 
for row in top_sports:
    print(row)

('fitness', 13)
('baseball', 10)
('swimming', 10)
('yoga', 7)
('climbing', 3)
('tennis', 3)
('basketball', 2)
('capoeira', 2)
('crossfit', 2)
('free_flying', 2)


### Top Tourism-related Amenties

In [20]:
top_tourism = conn.cursor()
t = ('tourism', 'tourism_1')
top_tourism.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value UNION SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC", t)

top_tourism = top_tourism.fetchall()
 
for row in top_tourism:
    print(row)

('picnic_site', 168)
('information', 75)
('hotel', 42)
('motel', 25)
('artwork', 23)
('attraction', 21)
('viewpoint', 20)
('museum', 12)
('camp_site', 4)
('gallery', 4)
('guest_house', 3)
('bus_stop', 1)
('caravan_site', 1)
('scenic_view', 1)
('theme_park', 1)
('viewpoint', 1)


### Top City Apprearance

In [21]:
top_city = conn.cursor()
t = ('city',)
top_city.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC LIMIT 10", t)

top_city = top_city.fetchall()
 
for row in top_city:
    print(row)

('Sunnyvale', 2734)
('San Jose', 361)
('Santa Clara', 89)
('Milpitas', 72)
('Los Gatos', 67)
('Saratoga', 49)
('Morgan Hill', 48)
('San Jos\xc3\xa9', 46)
('Cupertino', 29)
('Campbell', 21)


### Top Payment Method

In [22]:
top_payment = conn.cursor()
t = ('payment',)
top_payment.execute("SELECT key, COUNT(*) as num FROM nodes_tags WHERE type=? GROUP BY key ORDER BY num DESC", t)

top_payment = top_payment.fetchall()
 
for row in top_payment:
    print(row)

('bitcoin', 15)
('cash', 10)
('credit_cards', 9)
('litecoin', 5)
('coins', 3)
('none', 2)
('notes', 2)
('visa', 2)
('american_express', 1)
('diners_club', 1)
('discover_card', 1)
('dogecoin', 1)
('mastercard', 1)


## Additional Ideas

Looking at the 'nodes_tag' file, there seems to be a lot of inconsistencies or incorrect value.

As an open-sourced project, there isn't much restriction placed on data editing. As such, the integrity of the data is easily compromised when there are a lot of users.

Few recommended solutions that can be implemented are:
- Enforce compulsory columns where users have to fill in all columns in order to submit an entry
- Cross-referencing missing data from other open-sourced data from governmental agencies or API
- Restructure the schema of the databases so that users are clear what to enter (most useful for address where zipcode or state should have separate columns)
- Implement some sort of artificial intelligence to determine potential errors in the entries (such as overabbreviated street types)
- Have a bot to update/create entries using data from other sources

### Benefits

+ By having a more complete dataset, it can serve as a place to search for places or info for each city
+ With proper transportation related ways_tags data, public transportation data can be included
+ Cost of manually cleaning or editing errorneous data is high, and the use of a bot or machine learning might help to speed up and bring down the cost in the long run

### Anticipated Problems

- As an open-sourced platform, cost and human reousrces are main issues
- Lack of awareness of the existence OSM means it has to be self-sufficient and sustainable in its project taking

## Conclusion

The OpenStreetMap data is a great place for users to contribute to the data. However, if the integrity and completenenss of data are not maintained, it can't compete with other map services such as Google Maps or Apple Maps which has a better user experience and more structure ways of searching for info.

With more care taken, OpenStreetMap is capable of being the Wikipedia of maps.